In [1]:
import ast
import os
import pandas as pd
import re

pd.set_option("display.max_columns", None)

os.chdir("../")

# Belgium
- https://elections.fgov.be/elections-du-26-mai-2019-tableaux-des-resultats

In [2]:
year = 2019
election_type = "general"

In [3]:
df = pd.read_csv(f"data/Belgium/{year}_general.csv")\
    .rename(columns={"id": "polling_id"})
# df["electionLists"] = df["electionLists"].apply(lambda x: ast.literal_eval(x))
# df["labels"] = df["labels"].apply(lambda x: ast.literal_eval(x))
# df = df.explode("electionLists")
# df = df.explode("labels")

# A = pd.json_normalize(df["labels"])
# B = pd.json_normalize(df["electionLists"]).rename(columns={
#     "nrOfVotes": "value",
#     "partyLabel": "candidate"})

# df = df.join([A, B])

# print(list(df))

# df = df.drop(columns=["labels", "electionLists"])
df["year"] = df["electionDate"].str[0:4].astype(int)
df = df[df["year"] == year].copy()
# df = df.drop_duplicates()

df.head()
# df.shape

,polling_id,electionId,nrOfEligibleVoters,nrOfValidVotes,nrOfBlankVotes,nisCode,labels,description,electionType,electionDate,electionLists,year
1,229941,51204,0,20060,936,44084,"[{'language': 'French', 'label': 'Aalter'}, {'...",Municipality,Chamber,2019-05-26,"[{'id': 1325011, 'nrOfVotes': 2802, 'listNumbe...",2019
3,229779,51204,0,20639,1157,24001,"[{'language': 'French', 'label': 'Aarschot'}, ...",Municipality,Chamber,2019-05-26,"[{'id': 1323438, 'nrOfVotes': 3203, 'listNumbe...",2019
5,229656,51204,0,9855,255,11001,"[{'language': 'French', 'label': 'Aartselaar'}...",Municipality,Chamber,2019-05-26,"[{'id': 1321960, 'nrOfVotes': 1207, 'listNumbe...",2019
7,229778,51204,0,8951,332,23105,"[{'language': 'French', 'label': 'Affligem'}, ...",Municipality,Chamber,2019-05-26,"[{'id': 1323428, 'nrOfVotes': 1349, 'listNumbe...",2019
9,229981,51204,0,5867,678,52074,"[{'language': 'French', 'label': 'Aiseau-Presl...",Municipality,Chamber,2019-05-26,"[{'id': 1325516, 'nrOfVotes': 563, 'listNumber...",2019


In [4]:
len(df.polling_id.unique())

581

In [10]:
output = []
output_location = []
for polling_id, frag in df.groupby("polling_id"):
    # print(polling_id)

    parties = ast.literal_eval(frag["electionLists"].values[0])
    labels = ast.literal_eval(frag["labels"].values[0])

    commune = labels[1]["label"]

    output_location.append({
        "commune": commune,
        "polling_id": polling_id
    })

    for party in parties:
        # print(party)
        output.append({
            "polling_id": polling_id,
            "value": party["nrOfVotes"],
            "candidate": party["partyLabel"],
            "commune": commune
        })

df_location = pd.DataFrame(output_location)
df_filtered = pd.DataFrame(output)
df_filtered.head()

,polling_id,value,candidate,commune
0,229656,1207,open vld,Aartselaar
1,229656,4346,N-VA,Aartselaar
2,229656,1577,VLAAMS BELANG,Aartselaar
3,229656,859,CD&V,Aartselaar
4,229656,436,PVDA,Aartselaar


In [6]:
tt = df_filtered.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt.head()

,polling_id,candidate,value,rate
0,229656,CD&V,859,0.087164
1,229656,D-SA,9,0.000913
2,229656,DierAnimal,74,0.007509
3,229656,GROEN,727,0.073770
4,229656,N-VA,4346,0.440994


In [7]:
tt.to_csv(f"data_output/Belgium/{year}_{election_type}.csv.gz", compression="gzip", index=False)

In [8]:
df_lau = pd.read_excel(
    "https://ec.europa.eu/eurostat/documents/345175/501971/EU-27-LAU-2022-NUTS-2021.xlsx",
    sheet_name="BE"
)
df_lau = df_lau[["LAU NAME NATIONAL", "NUTS 3 CODE", "LAU CODE"]].rename(columns={"LAU NAME NATIONAL": "commune", "NUTS 3 CODE": "nuts_3", "LAU CODE": "lau_id"})
df_lau["nuts_2"] = df_lau["nuts_3"].str[0:4]
df_lau["nuts_1"] = df_lau["nuts_3"].str[0:3]
df_lau["lau_id"] = df_lau["lau_id"].astype(str).str.zfill(8)
df_lau["commune"] = df_lau["commune"].str.replace("’", "'")
df_lau["commune"] = df_lau["commune"].apply(lambda x: re.sub(r"\([^)]*\)", "", x)).str.strip()

if year == 2019:
    df_lau["commune"] = df_lau["commune"].replace({
        "'s Gravenbrakel": "s Gravenbrakel",
        "Celles": "Celles (lez-Tournai)",
        "Eigenbrakel": "Braine-l'Alleud",
        "Meix-devant-Virton": "Meix-Devant-Virton",
        "Opzullik": "Silly",
        "Saint-Nicolas": "Saint-Nicolas (Liège)",
        "Sint-Joost-ten-Node": "Sint-Joost-Ten-Node",
        "Villers-Le-Bouillet": "Villers-le-Bouillet"
    })
elif year == 2014:
    df_lau["commune"] = df_lau["commune"].replace({
        "'s Gravenbrakel": "s Gravenbrakel",
        "Celles": "Celles (lez-Tournai)",
        "Eigenbrakel": "Braine-l'Alleud",
        "Meix-devant-Virton": "Meix-Devant-Virton",
        "Opzullik": "Silly",
        "Sint-Joost-ten-Node": "Sint-Joost-Ten-Node",
        "Villers-Le-Bouillet": "Villers-le-Bouillet",
        'Aiseau-Presles': "Aiseau-Presles Nl",
        "De Haan": "de Haan",
        "Glabbeek": "Glabeek",
        "Kruisem": "Kruisem",
        "La Bruyère": "la Bruyère",
        "Lievegem": "Lievegem",
        "Montigny-le-Tilleul": "Montigne-le-Tilleul",
        "Oudsbergen": "Oudsbergen",
        "Pelt": "Pelt",
        "Puurs-Sint-Amands": "Puurs-Sint-Amands",
        "Saint-Nicolas": "Saint-Nicolas (liège)"
    })

df_lau.head()

RemoteDisconnected: Remote end closed connection without response

In [ ]:
set(df_lau.commune) - set(df_location.commune)

{'Kruisem', 'Lievegem', 'Oudsbergen', 'Pelt', 'Puurs-Sint-Amands'}

In [ ]:
set(df_location.commune) - set(df_lau.commune)

set()

In [ ]:
df_location = pd.merge(df_location, df_lau, on="commune", how="left")

df_location.to_csv(f"data_output/Belgium/{year}_{election_type}_location.csv.gz", compression="gzip", index=False)